In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.naive_bayes import GaussianNB    
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# data loading
data = pd.read_csv(r'''D:\A309-21-\课题\DILI Predict\DILI数据+模型\汇总-DILI\train\Padel-MACCSdata.csv''', low_memory=False)
X = data.iloc[:,3:].values
y = data.loc[:,'DILIst.1'].values
# fixed random seed so that training is the same for each division, shuffle: reorder the data before splitting
data_loader = DataLoader(dataset=data, batch_size=8, shuffle=True, num_workers=0, drop_last=False)
dataset = data.sample(frac=1).reset_index(drop=True) 
# slicing the dataset in two
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=78, shuffle=True)

In [ ]:
# gnb
gnb = GaussianNB(priors = None, var_smoothing = 1e-03)
# cross-validation
gnb_s = cross_val_score(gnb, X_train, y_train, cv = 5, scoring = "precision_weighted").mean()
print("GNB:{}".format(gnb_s))
gnb.fit(X_train, y_train)
# evaluation on test set
y_score_test = gnb.predict(X_test).tolist()     
y_pred_test = (gnb.predict(X_test) > 0.5).astype("int32")      
TN, FP, FN, TP = confusion_matrix(y_test, y_pred_test).ravel()   
SE = TP / (TP + FN)
SP = TN / (TN + FP)
ACC = (TP + TN) / (TP + TN + FP + FN)                    
F1 = (TP / (2 * TP + FN + FP)) * 2
MCC = (TP * TN - FP * FN) / ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) ** 0.5
AUC = roc_auc_score(y_test, y_score_test)
BA = balanced_accuracy_score(y_test, y_pred_test)
LR = pd.DataFrame({'SE': [SE], 'SP': [SP], 'ACC': [ACC], 'AUC': [AUC], 'F1': [F1], 'MCC': [MCC], 'BA': [BA]})
print(LR)